In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import KFold
from IPython.display import HTML, display
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


pd.options.display.max_rows = 1000
pd.options.display.max_columns = 20


In [ ]:
import warnings

# 경고 무시하기
warnings.filterwarnings("ignore")
# unicode minus를 사용하지 않기 위한 설정 (minus 깨짐현상 방지)
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'Malgun Gothic'

In [ ]:
df = pd.read_csv('강남구전세데이터_찐마지막.csv')

In [ ]:
print(df.info())

In [ ]:
print(df.describe())
print(df.describe(include='object'))

In [ ]:
# (선택) 결측치 확인
print(df.isnull().sum())

# 결측치 처리 (예시: 평균값으로 대체)
df.fillna(df.mean(), inplace=True)

In [ ]:
df

In [ ]:
# 'y' 값이 가장 큰 상위 100개의 데이터를 선택
top_100 = df.nlargest(100, 'y')

# 상위 100개 데이터의 시각화 예시
plt.figure(figsize=(10, 6))
sns.scatterplot(data=top_100, x='Longitude', y='Latitude', hue='y', palette='coolwarm')
plt.title('Top 100 Properties by y Value')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend(title='y')
plt.show()

In [ ]:
pip install folium

In [ ]:
import folium

# 상위 100개 데이터 중심 좌표 계산
center_lat = top_100['Latitude'].mean()
center_lon = top_100['Longitude'].mean()

# Folium 지도 객체 생성
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# 상위 100개 데이터에 대한 마커 추가
for idx, row in top_100.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"y: {row['y']}",
        icon=folium.Icon(color='blue')
    ).add_to(m)

# 지도를 HTML 파일로 저장
m.save('top_100_map.html')

In [ ]:
import folium

# 상위 100개 데이터 중심 좌표 계산
center_lat = top_100['Latitude'].mean()
center_lon = top_100['Longitude'].mean()

# Folium 지도 객체 생성
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# 상위 100개 데이터에 대한 마커 추가
for idx, row in top_100.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"y: {row['y']}",
        icon=folium.Icon(color='blue')
    ).add_to(m)

# 주피터 노트북에 지도 표시
m

In [ ]:
from folium.plugins import MarkerCluster

# Folium 지도 객체 생성
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# MarkerCluster 객체 생성
marker_cluster = MarkerCluster().add_to(m)

# 상위 100개 데이터에 대한 CircleMarker 추가
for idx, row in top_100.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,
        popup=f"y: {row['y']}",
        color='blue',  # 마커의 색상은 여기서는 일정하게 지정
        fill_color='blue',
    ).add_to(marker_cluster)

# 주피터 노트북에 지도 표시
m

In [ ]:
from folium.plugins import HeatMap

# Folium 지도 객체 생성
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# 데이터를 (위도, 경도) 형태로 변환
data = top_100[['Latitude', 'Longitude']].values.tolist()

# Heatmap 추가
HeatMap(data, radius=15).add_to(m)

# 주피터 노트북에 지도 표시
m

In [ ]:
import geopandas as gpd
import pandas as pd
import os

path = 'emd_20230729'
emd = gpd.read_file(os.path.join(path, 'emd.shp'), encoding='cp949')

emd.crs="EPSG:5179"

emd['center_point'] = emd['geometry'].geometry.centroid # 선 중심
emd['geometry'] = emd['geometry'].to_crs(epsg=4326) # 후 변환
emd['center_point'] = emd['center_point'].to_crs(epsg=4326)
emd['경도'] = emd['center_point'].map(lambda x: x.xy[0][0])
emd['위도'] = emd['center_point'].map(lambda x: x.xy[1][0])

geo_j = emd[emd['EMD_KOR_NM'].isin(df['Admin_District'].unique())].drop(['center_point'], axis=1).to_json()

m = folium.Map(location=[df['Latitude'].mean() , df['Longitude'].mean()], zoom_start=12.5)

folium.GeoJson(
    geo_j,
    name='지역구'
).add_to(m)

m


In [ ]:
import geopandas as gpd
import pandas as pd
import os
import folium
from folium.plugins import HeatMap

# GeoDataFrame 생성
path = 'emd_20230729'
emd = gpd.read_file(os.path.join(path, 'emd.shp'), encoding='cp949')

emd.crs = "EPSG:5179"

emd['center_point'] = emd['geometry'].geometry.centroid
emd['geometry'] = emd['geometry'].to_crs(epsg=4326)
emd['center_point'] = emd['center_point'].to_crs(epsg=4326)
emd['경도'] = emd['center_point'].map(lambda x: x.xy[0][0])
emd['위도'] = emd['center_point'].map(lambda x: x.xy[1][0])

# Folium 지도 객체 생성
m = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=12.5)

# 지역구 경계 데이터 표시
for idx, row in emd.iterrows():
    if row['geometry'].geom_type == 'Polygon':
        folium.Polygon(
            locations=row['geometry'].exterior.coords,
            color='black',
            fill_color='white',
            fill_opacity=0.3,
            weight=1
        ).add_to(m)
    elif row['geometry'].geom_type == 'MultiPolygon':
        for polygon in row['geometry'].geoms:
            folium.Polygon(
                locations=polygon.exterior.coords,
                color='black',
                fill_color='white',
                fill_opacity=0.3,
                weight=1
            ).add_to(m)

# 데이터를 (위도, 경도) 형태로 변환
data = top_100[['Latitude', 'Longitude']].values.tolist()

# Heatmap 추가
HeatMap(data, radius=15).add_to(m)

# GeoJSON 지역구 경계 추가
geo_j = emd[emd['EMD_KOR_NM'].isin(df['Admin_District'].unique())].drop(['center_point'], axis=1).to_json()
folium.GeoJson(
    geo_j,
    name='지역구'
).add_to(m)

# 주피터 노트북에 지도 표시
m